In [1]:
from pydantic import BaseModel
from typing import Optional


class RFDiffusionScriptConfig(BaseModel):
    # Required fields
    output_prefix: str  # e.g., "example_outputs/design_motifscaffolding_with_target" or "example_outputs/design_motifscaffolding_inpaintseq"
    input_pdb: str      # e.g., "input_pdbs/1YCR.pdb" or "input_pdbs/5TPN.pdb"
    contigmap_contigs: str  # e.g., "[A25-109/0 0-70/B17-29/0-70]" or "[10-40/A163-181/10-40]"
    num_designs: int

    # Optional fields
    contigmap_length: Optional[str] = None  # e.g., "70-120"
    contigmap_inpaint_seq: Optional[str] = None  # e.g., "[A163-168/A170-171/A179]"
    ckpt_override_path: Optional[str] = None  # e.g., "../models/Complex_base_ckpt.pt"

    def get_script(self) -> str:
        """
        Constructs and returns the bash command string for running the RF Diffusion inference.
        """
        # Start with the Python script call and the base prefix arguments
        cmd_parts = [
            "python ../scripts/run_inference.py",
            f"inference.output_prefix={self.output_prefix}",
            f"inference.input_pdb={self.input_pdb}",
            f"'contigmap.contigs={self.contigmap_contigs}'",
            f"inference.num_designs={self.num_designs}",
        ]

        # Conditionally add the contigmap length parameter
        if self.contigmap_length:
            cmd_parts.insert(3, f"contigmap.length={self.contigmap_length}")

        # Conditionally add the inpaint sequence parameter
        if self.contigmap_inpaint_seq:
            cmd_parts.append(f"'contigmap.inpaint_seq={self.contigmap_inpaint_seq}'")

        # Conditionally add the checkpoint override if provided
        if self.ckpt_override_path:
            cmd_parts.append(f"inference.ckpt_override_path={self.ckpt_override_path}")

        # Join all parts into one command string with a space separator and return it
        return " ".join(cmd_parts)


# Example usage:

# Example for the first script variant (p53-Mdm2 motif-scaffolding)
config1 = RFDiffusionScriptConfig(
    output_prefix="example_outputs/design_motifscaffolding_with_target",
    input_pdb="input_pdbs/1YCR.pdb",
    contigmap_contigs="[A25-109/0 0-70/B17-29/0-70]",
    contigmap_length="70-120",
    num_designs=10,
    ckpt_override_path="../models/Complex_base_ckpt.pt"
)

# Example for the second script variant (RSV-F with inpainting)
config2 = RFDiffusionScriptConfig(
    output_prefix="example_outputs/design_motifscaffolding_inpaintseq",
    input_pdb="input_pdbs/5TPN.pdb",
    contigmap_contigs="[10-40/A163-181/10-40]",
    num_designs=10,
    contigmap_inpaint_seq="[A163-168/A170-171/A179]"
)

# Retrieving the script as a string:
print("Script for p53-Mdm2 motif-scaffolding:")
print(config1.get_script())
print("\nScript for RSV-F with inpaint sequence:")
print(config2.get_script())


Script for p53-Mdm2 motif-scaffolding:
python ../scripts/run_inference.py inference.output_prefix=example_outputs/design_motifscaffolding_with_target inference.input_pdb=input_pdbs/1YCR.pdb contigmap.length=70-120 'contigmap.contigs=[A25-109/0 0-70/B17-29/0-70]' inference.num_designs=10 inference.ckpt_override_path=../models/Complex_base_ckpt.pt

Script for RSV-F with inpaint sequence:
python ../scripts/run_inference.py inference.output_prefix=example_outputs/design_motifscaffolding_inpaintseq inference.input_pdb=input_pdbs/5TPN.pdb 'contigmap.contigs=[10-40/A163-181/10-40]' inference.num_designs=10 'contigmap.inpaint_seq=[A163-168/A170-171/A179]'


In [3]:
from pydantic import BaseModel, Field
from typing import Optional

class RFDiffusionScriptConfig(BaseModel):
    """
    Data model for configuring an RF Diffusion script for protein motif scaffolding.
    
    Attributes:
        output_prefix: The directory and filename prefix where design outputs will be saved.
        input_pdb: File path to the input PDB containing the protein complex structure.
        contigmap_contigs: A string defining the fixed/flexible design regions (contigs). 
                           For example, "[A25-109/0 0-70/B17-29/0-70]" or "[10-40/A163-181/10-40]".
        num_designs: Number of design variants to generate.
        contigmap_length: (Optional) Constraint for total design length (e.g., "70-120").
        contigmap_inpaint_seq: (Optional) Specifies residues within a contig to allow inpainting 
                               (i.e., to be redesigned), e.g., "[A163-168/A170-171/A179]".
        ckpt_override_path: (Optional) File path to a model checkpoint to override the default.
    """
    
    output_prefix: str = Field(
        ...,
        description="Output directory and file prefix for design results (e.g., 'example_outputs/design_motifscaffolding_with_target')."
    )
    input_pdb: str = Field(
        ...,
        description="Path to the input PDB file containing the protein complex structure (e.g., 'input_pdbs/1YCR.pdb')."
    )
    contigmap_contigs: str = Field(
        ...,
        description="Defines the fixed and flexible regions for design, such as '[A25-109/0 0-70/B17-29/0-70]' for p53-Mdm2 or '[10-40/A163-181/10-40]' for RSV-F."
    )
    num_designs: int = Field(
        ...,
        description="Number of design variants to generate."
    )
    contigmap_length: Optional[str] = Field(
        None,
        description="Optional total length constraints for the design (e.g., '70-120')."
    )
    contigmap_inpaint_seq: Optional[str] = Field(
        None,
        description="Optional flag specifying which residues within a contig should be redesigned (inpainted), such as '[A163-168/A170-171/A179]'."
    )
    ckpt_override_path: Optional[str] = Field(
        None,
        description="Optional path to a model checkpoint to override the default (e.g., '../models/Complex_base_ckpt.pt')."
    )
    
    def get_script(self) -> str:
        """
        Constructs and returns the complete bash command string for the RF Diffusion inference.
        
        Returns:
            A string containing the command that runs the inference with the configuration parameters.
        """
        # Base command with essential parameters
        cmd_parts = [
            "../scripts/run_inference.py",
            f"inference.output_prefix={self.output_prefix}",
            f"inference.input_pdb={self.input_pdb}",
            f"'contigmap.contigs={self.contigmap_contigs}'",
            f"inference.num_designs={self.num_designs}",
        ]
        
        # Insert contigmap length if provided (positioned after input PDB)
        if self.contigmap_length:
            cmd_parts.insert(3, f"contigmap.length={self.contigmap_length}")
        
        # Append inpaint sequence if provided
        if self.contigmap_inpaint_seq:
            cmd_parts.append(f"'contigmap.inpaint_seq={self.contigmap_inpaint_seq}'")
        
        # Append checkpoint override if provided
        if self.ckpt_override_path:
            cmd_parts.append(f"inference.ckpt_override_path={self.ckpt_override_path}")
        
        # Combine all command parts into a single string
        return " ".join(cmd_parts)

# Example usage for the p53-Mdm2 motif-scaffolding variant:
config1 = RFDiffusionScriptConfig(
    output_prefix="example_outputs/design_motifscaffolding_with_target",
    input_pdb="input_pdbs/1YCR.pdb",
    contigmap_contigs="[A25-109/0 0-70/B17-29/0-70]",
    contigmap_length="70-120",
    num_designs=10,
    ckpt_override_path="../models/Complex_base_ckpt.pt"
)

# Example usage for the RSV-F motif with inpainting variant:
config2 = RFDiffusionScriptConfig(
    output_prefix="example_outputs/design_motifscaffolding_inpaintseq",
    input_pdb="input_pdbs/5TPN.pdb",
    contigmap_contigs="[10-40/A163-181/10-40]",
    num_designs=10,
    contigmap_inpaint_seq="[A163-168/A170-171/A179]"
)

# Retrieve and print the generated script commands
print("Script for p53-Mdm2 motif-scaffolding:")
print(config1.get_script())
print("\nScript for RSV-F with inpaint sequence:")
print(config2.get_script())


Script for p53-Mdm2 motif-scaffolding:
../scripts/run_inference.py inference.output_prefix=example_outputs/design_motifscaffolding_with_target inference.input_pdb=input_pdbs/1YCR.pdb contigmap.length=70-120 'contigmap.contigs=[A25-109/0 0-70/B17-29/0-70]' inference.num_designs=10 inference.ckpt_override_path=../models/Complex_base_ckpt.pt

Script for RSV-F with inpaint sequence:
../scripts/run_inference.py inference.output_prefix=example_outputs/design_motifscaffolding_inpaintseq inference.input_pdb=input_pdbs/5TPN.pdb 'contigmap.contigs=[10-40/A163-181/10-40]' inference.num_designs=10 'contigmap.inpaint_seq=[A163-168/A170-171/A179]'
